# VT Graphs in Jupyter Notebook

In this notebook we will explore how to obtain attributes and relationship for different entities using VirusTotal API v3. Finally we can render all the relationships we have obtained using VTGraph.

## Import libraries

In [1]:
from msticpy.sectools.vtlookupv3 import VTLookupV3

import networkx as nx
import matplotlib.pyplot as plt
import os
import pandas as pd

import nest_asyncio
nest_asyncio.apply()

## Create Lookup instance

In [2]:
# Obtain key from env varaible
vt_key = os.environ["VT_API_KEY"]
# Instanciate vt_lookup object
vt_lookup = VTLookupV3(vt_key)

## Types of DataFrames

Our functions will return and accept two different types of DataFrames.

- **Attributes DataFrame**: Contains the properties of a entity. It has the columns:
    - id
    - type
    - ..properties
- **Relationships DataFrames**: Contains the relationship between two entities, and the type of relationship. It contains the columns:
    - source
    - target
    - source_type
    - target_type
    - relationship_type

In [3]:
FILE = 'ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa'

In [4]:
example_attribute_df = vt_lookup.lookup_ioc(observable=FILE, vt_type='file')
example_attribute_df

,type_description,size,first_submission_date,times_submitted,meaningful_name,last_submission_date,detections,scans,type
id,,,,,,,,,
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa,Win32 EXE,3514368,1494574270,1455,diskpart.exe,1597190684,65,73,file


In [5]:
example_relationship_df = vt_lookup.lookup_ioc_relationships(
    observable=FILE, 
    vt_type='file', 
    relationship='execution_parents')
example_relationship_df

target_type  \
source                                             target                                                           
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61...        file   
                                                   32f24601153be0885f11d62e0a8a2f0280a2034fc981d81...        file   
                                                   bd927d915f19a89468391133465b1f2fb78d7a581788679...        file   
                                                   9f35e3393e442b4a35422e4b927e530712bac8ee2034a88...        file   
                                                   f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dc...        file   
...                                                                                                           ...   
                                                   40bc59671366a138b1df704f4419521dd8983b46e026afc...        file   
                                                   64488ed709c5ca9bf23bf943711e85faea008b341fbf662...        file   
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...        file   
                                                   9a2e1e8da634aa1cce446a9ee8f4de2a357eb76debf2793...        file   
                                                   36dadfcae878c299ea1b563eac0240a7b352a0ff24c5992...        file   

                                                                                                      source_type  \
source                                             target                                                           
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61...        file   
                                                   32f24601153be0885f11d62e0a8a2f0280a2034fc981d81...        file   
                                                   bd927d915f19a89468391133465b1f2fb78d7a581788679...        file   
                                                   9f35e3393e442b4a35422e4b927e530712bac8ee2034a88...        file   
                                                   f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dc...        file   
...                                                                                                           ...   
                                                   40bc59671366a138b1df704f4419521dd8983b46e026afc...        file   
                                                   64488ed709c5ca9bf23bf943711e85faea008b341fbf662...        file   
                                                   06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e52...        file   
                                                   9a2e1e8da634aa1cce446a9ee8f4de2a357eb76debf2793...        file   
                                                   36dadfcae878c299ea1b563eac0240a7b352a0ff24c5992...        file   

                                                                                                       relationship_type  
source                                             target                                                                 
ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c... 24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61...  execution_parents  
                                                   32f24601153be0885f11d62e0a8a2f0280a2034fc981d81...  execution_parents  
                                                   bd927d915f19a89468391133465b1f2fb78d7a581788679...  execution_parents  
                                                   9f35e3393e442b4a35422e4b927e530712bac8ee2034a88...  execution_parents  
                                                   f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dc...  execution_parents  
...                                                                                                                  ...  
                                                   40bc59671366a138b1df704

### Obtaining result for multiple entities

The function `lookup_iocs` is able to obtain attributes for all the rows in a DataFrame. If no `observable_column` and `observable_type` parameters are specified, the function will obtain the attributes of all the entities that are in the column `target`, and will obtain their types from the `target_type` column.

This function is especially usefull when a user has obtained a set of relationships, and would like to obtain their attributes.

In [6]:
example_multiple_attribute_df = vt_lookup.lookup_iocs(example_relationship_df)
example_multiple_attribute_df

,type_description,size,first_submission_date,times_submitted,meaningful_name,last_submission_date,detections,scans,type
id,,,,,,,,,
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa614ea04703480b1022c,Win32 EXE,3723264,1494579471,250,lhdfrgui.exe,1591271694,65,73,file
32f24601153be0885f11d62e0a8a2f0280a2034fc981d8184180c5d3b1b9e8cf,Win32 EXE,3723264,1494763431,59,lhdfrgui.exe,1583504867,64,73,file
bd927d915f19a89468391133465b1f2fb78d7a58178867933c44411f4d5de8eb,Win32 EXE,3723264,1494796916,18,lhdfrgui.exe,1577642121,67,73,file
9f35e3393e442b4a35422e4b927e530712bac8ee2034a884b6ef724f0d8ec0ef,Win32 EXE,3723264,1494831892,12,lhdfrgui.exe,1577647004,68,74,file
f0105b922f41b0ee8595e0e7d989c9ba69d4a38337211dcbe86e0bdce346853d,Win32 EXE,3723264,1494880301,9,lhdfrgui.exe,1571667977,66,71,file
...,...,...,...,...,...,...,...,...,...
40bc59671366a138b1df704f4419521dd8983b46e026afcce5869c51c223ef65,Win32 EXE,3967856,1519276624,3,KSkill.exe,1519276624,40,73,file
64488ed709c5ca9bf23bf943711e85faea008b341fbf662c8982649240aa8203,Win32 EXE,4315347,1594903025,2,ransom.exe,1594903025,48,77,file
06c676bf8f5c6af99172c1cf63a84348628ae3f39df9e523c42447e2045e00ff,Win32 EXE,4535704,1595479073,1,car.exe,1595479073,40,75,file


Also, if we would like to obtain the relationships for a set of entities, we have the function `lookup_iocs_relationships`. Here also, if no `observable_column` and `observable_type` parameters are specified, the function will obtain the relationships of all the entities that are in the column `target`, and will obtain their types from the `target_type` column.

In [7]:
example_multiple_relationship_df = vt_lookup.lookup_iocs_relationships(example_relationship_df, 'contacted_domains')
example_multiple_relationship_df

target_type  \
source                                             target                                                           
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com       domain   
32f24601153be0885f11d62e0a8a2f0280a2034fc981d81... www.ifferfsodp9ifjaposdfjhgosurijfaewrwergwea.com       domain   
bd927d915f19a89468391133465b1f2fb78d7a581788679... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.test      domain   
9f35e3393e442b4a35422e4b927e530712bac8ee2034a88... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.cum       domain   
1e06140672b73dfe337dfde7bc9dead5612bdbf4a8069be... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com       domain   
...                                                                                                           ...   
64488ed709c5ca9bf23bf943711e85faea008b341fbf662... wu.ec.azureedge.net                                     domain   
                                                   wu.wpc.apr-52dd2.edgecastdns.net                        domain   
                                                   a1363.dscg.akamai.net                                   domain   
                                                   crl.www.ms.akadns.net                                   domain   
9a2e1e8da634aa1cce446a9ee8f4de2a357eb76debf2793... dmitri.duckdns.org                                      domain   

                                                                                                      source_type  \
source                                             target                                                           
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com         file   
32f24601153be0885f11d62e0a8a2f0280a2034fc981d81... www.ifferfsodp9ifjaposdfjhgosurijfaewrwergwea.com         file   
bd927d915f19a89468391133465b1f2fb78d7a581788679... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.test        file   
9f35e3393e442b4a35422e4b927e530712bac8ee2034a88... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.cum         file   
1e06140672b73dfe337dfde7bc9dead5612bdbf4a8069be... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com         file   
...                                                                                                           ...   
64488ed709c5ca9bf23bf943711e85faea008b341fbf662... wu.ec.azureedge.net                                       file   
                                                   wu.wpc.apr-52dd2.edgecastdns.net                          file   
                                                   a1363.dscg.akamai.net                                     file   
                                                   crl.www.ms.akadns.net                                     file   
9a2e1e8da634aa1cce446a9ee8f4de2a357eb76debf2793... dmitri.duckdns.org                                        file   

                                                                                                       relationship_type  
source                                             target                                                                 
24d004a104d4d54034dbcffc2a4b19a11f39008a575aa61... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com   contacted_domains  
32f24601153be0885f11d62e0a8a2f0280a2034fc981d81... www.ifferfsodp9ifjaposdfjhgosurijfaewrwergwea.com   contacted_domains  
bd927d915f19a89468391133465b1f2fb78d7a581788679... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.test  contacted_domains  
9f35e3393e442b4a35422e4b927e530712bac8ee2034a88... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.cum   contacted_domains  
1e06140672b73dfe337dfde7bc9dead5612bdbf4a8069be... www.iuqerfsodp9ifjaposdfjhgosurijfaewrwergwea.com   contacted_domains  
...                                                                                                                  ...  
64488ed709c5ca9bf23bf943711e85faea008b341fbf662... wu.ec.azureedge.net    

## Integration with VTGraph

Once we have some DataFrames with the relationships, we are able to generate and visualize a VT Graph in our notebook. The function `create_vt_graph` accepts as input a **list of Relationship DataFrames**.

In [8]:
graph_id = vt_lookup.create_vt_graph(
    relationship_dfs=[example_relationship_df, example_multiple_relationship_df],
    name="My first Jupyter Notebook Graph"
)
graph_id

'g7d2ad7259f6e415889f0b9da9ab9528f065971174ba14e328b877138c3f7e361'

In [10]:
vt_lookup.render_vt_graph(
    graph_id = graph_id,
    width = 800,
    height = 500
)